# Industry 4.0 의 중심, BigData

<div align='right'><font size=2 color='gray'>Data Processing Based Python @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, [김진수](bigpycraft@gmail.com)</font></div>
<hr>

## 웹 크롤링 실습

### 위키피디어 정보 크롤링
<div align='left'>
<table width=100%>
    <tr>
        <td width=200>
<img alt="Vivaldi.jpg" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Vivaldi.jpg/250px-Vivaldi.jpg" decoding="async" width="100" height="120">
        </td>
        <td>
            <div align="left">
<ul> 
    <br/><li>주제키워드 : <a href='https://ko.wikipedia.org/wiki/%EC%95%88%ED%86%A0%EB%8B%88%EC%98%A4_%EB%B9%84%EB%B0%9C%EB%94%94'>안토니오 비발디</a></li>
    <br/><li>인물 기본정보 : 출생, 사망, 직업, 장르, 악기, 활동시기, 배우자, 종교</li>
    <br/><li>미디어 리스트 : list [ (작품제목, 작품설명) ]</li>
    <br/><li>수집된 자료 파일로 저장</li>
</ul>    
            </div>
        </td>
    </tr>
</table>
</div>

#### <font color='#0000CC'>Step1 : 웹페이지 정보를 읽어오기</font>

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen
from urllib import parse

from IPython.display import Image 
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
# 구글에서 '50 Best Sandwiches' 검색
keyword = '안토니오_비발디'
url = 'https://ko.wikipedia.org/wiki/{}'.format(parse.quote(keyword))
url

'https://ko.wikipedia.org/wiki/%EC%95%88%ED%86%A0%EB%8B%88%EC%98%A4_%EB%B9%84%EB%B0%9C%EB%94%94'

In [3]:
html = urlopen(url)
soup = BeautifulSoup(html, "lxml")
soup.title

<title>안토니오 비발디 - 위키백과, 우리 모두의 백과사전</title>

#### <font color='#CC0000'>기본정보 수집</font>

In [4]:
infobox_table = soup.find('table', 'infobox vcard')
infobox_table

<table cellspacing="3" class="infobox vcard" style="border-spacing: 3px; width:300px; text-align:left"><tbody><tr><th class="summary" colspan="2" style="text-align:center; font-size:125%; font-weight:bold; background:silver;"><div style="position:relative; width:100%; height:34px; display:table;"><div style="position:absolute; top:0; right:0;"><img alt="Picto infobox music.png" data-file-height="45" data-file-width="44" decoding="async" height="34" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/60/Picto_infobox_music.png/33px-Picto_infobox_music.png" srcset="//upload.wikimedia.org/wikipedia/commons/6/60/Picto_infobox_music.png 1.5x" width="33"/></div><span style="position:relative; width:100%; display:table-cell; vertical-align:middle;">안토니오 비발디<br/>Antonio Lucio Vivaldi</span></div></th></tr><tr class=""><td class="" colspan="2" style="text-align:center;">
<a class="image" href="/wiki/%ED%8C%8C%EC%9D%BC:Vivaldi.jpg"><img alt="Vivaldi.jpg" data-file-height="1533" data-file-width

In [5]:
infobox_tr = infobox_table.find_all('tr')
len(infobox_tr)

11

In [6]:
tr_num = 0
for tr_tag in infobox_tr:
    tr_num += 1
    tr_text = tr_tag.get_text()
    th_cnt  = len(tr_tag.find_all('th'))
    td_cnt  = len(tr_tag.find_all('td'))
    print("line {} ({},{}): {}".format(tr_num, th_cnt, td_cnt, tr_text))
    print('-'*30)

line 1 (1,0): 안토니오 비발디Antonio Lucio Vivaldi
------------------------------
line 2 (0,1): 

------------------------------
line 3 (1,0): 기본 정보
------------------------------
line 4 (1,1): 출생

1678년 3월 4일(1678-03-04) 이탈리아 베네치아
------------------------------
line 5 (1,1): 사망

1741년 7월 28일(1741-07-28) (63세) 이탈리아 베네치아
------------------------------
line 6 (1,1): 직업

바이올린 연주가, 작곡가
------------------------------
line 7 (1,1): 장르

클래식 음악
------------------------------
line 8 (1,1): 악기

바이올린
------------------------------
line 9 (1,1): 활동 시기

1701년 ~ 1741년
------------------------------
line 10 (1,1): 배우자

無
------------------------------
line 11 (1,1): 종교

천주교(가톨릭 사제)
------------------------------


In [7]:
tr_num = 0

info_dict = dict()
info_key = ''
info_val = ''

for tr_tag in infobox_tr:
    tr_num += 1
    tr_text = tr_tag.get_text()
    th_cnt  = len(tr_tag.find_all('th'))
    td_cnt  = len(tr_tag.find_all('td'))
    
    if th_cnt>0 and td_cnt>0:
        info_key = tr_tag.find('th').get_text().strip()
        info_val = tr_tag.find('td').get_text().strip()
        info_dict[info_key] = info_val
        
        # print("line {} = '{}': '{}' ".format(tr_num, info_key, info_val))

info_dict        

{'출생': '1678년 3월 4일(1678-03-04) 이탈리아 베네치아',
 '사망': '1741년 7월 28일(1741-07-28) (63세) 이탈리아 베네치아',
 '직업': '바이올린 연주가, 작곡가',
 '장르': '클래식 음악',
 '악기': '바이올린',
 '활동 시기': '1701년 ~ 1741년',
 '배우자': '無',
 '종교': '천주교(가톨릭 사제)'}

In [8]:
info_s = Series(info_dict, name='기본정보')
info_s

출생              1678년 3월 4일(1678-03-04) 이탈리아 베네치아
사망       1741년 7월 28일(1741-07-28) (63세) 이탈리아 베네치아
직업                                  바이올린 연주가, 작곡가
장르                                         클래식 음악
악기                                           바이올린
활동 시기                               1701년 ~ 1741년
배우자                                             無
종교                                    천주교(가톨릭 사제)
Name: 기본정보, dtype: object

#### <font color='#CC0000'>미디어 리스트 수집</font>

In [9]:
media_div = soup.find('div', 'medialist listenlist noprint')
# media_div

In [10]:
media_list = media_div.find_all('table')
len(media_list)

19

In [11]:
media_tr = media_list[0].find_all('tr')
len(media_tr)

3

In [12]:
for idx in range(len(media_tr)):
    tr_text = media_tr[idx].get_text().strip()
    print(tr_text)

Op. 4 Concerto 1 "Spring" mvt 1: Allegro

비발디의 사계 중 봄 1악장 알레그로, 요한 해리슨 바이올린


In [13]:

def getMediaList(soup):
    media_op_list = list()
    media_kr_list = list()

    media_div = soup.find('div', 'medialist listenlist noprint')
    media_list = media_div.find_all('table')
    
    for idx in range(len(media_list)):
        media_tr = media_list[idx].find_all('tr')
        
        text_op = media_tr[0].get_text().strip()
        text_kr = media_tr[2].get_text().strip()
        
        media_op_list.append(text_op)
        media_kr_list.append(text_kr)
        
    return media_op_list, media_kr_list



In [14]:
media_op_list, media_kr_list = getMediaList(soup)

In [15]:
media_op_list

['Op. 4 Concerto 1 "Spring" mvt 1: Allegro',
 'Op. 4 Concerto 1 "Spring" mvt 2: Largo',
 'Op. 4 Concerto 1 "Spring" mvt 3: Allegro',
 'Op. 4 Concerto 2 "Summer" mvt 1: Allegro non molto',
 'Op. 4 Concerto 2 "Summer" mvt 2: Adagio',
 'Op. 4 Concerto 2 "Summer" mvt 3: Presto',
 'Op. 4 Concerto 3 "Autumn" mvt 1: Allegro',
 'Op. 4 Concerto 3 "Autumn" mvt 2: Adagio molto',
 'Op. 4 Concerto 3 "Autumn" mvt 3: Allegro',
 'Op. 4 Concerto 4 "Winter" mvt 1: Allegro non molto',
 'Op. 4 Concerto 4 "Winter" mvt 2: Largo',
 'Op. 4 Concerto 4 "Winter" mvt 3: Allegro',
 'Concerto for Oboe and Orchestra in C major (RV 447) - 2. Andante',
 'Credo (RV 591) - 1. Credo in Unum Deum',
 'Credo (RV 591) - 2. Et Incarnatus Est',
 'Credo (RV 591) - 3. Crucifixus',
 'Nulla in mundo pax sincera (RV 630) - 1. Larghetto',
 'Nulla in mundo pax sincera (RV 630) - 2. Recitativ',
 'Nulla in mundo pax sincera (RV 630) - 3. Allegro']

In [16]:
media_kr_list

['비발디의 사계 중 봄 1악장 알레그로, 요한 해리슨 바이올린',
 '비발디의 사계 중 봄 2악장 라르고, 요한 해리슨 바이올린',
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 "From Vivaldi's Four Seasons. John Harrison, Violin",
 'Performed by the Advent Chamber Orchestra with oboeist                 Edino Biaggi',
 'Performed by the Advent Chamber Orchestra with the              Advent Choir directed by Gabriel Pavel',
 'Performed by the Advent Chamber Orchestra with the              Advent Choir directed by Gabriel Pavel',
 'Performed by the Advent Chamber Orchestra with the              Advent 

In [17]:
media_df = DataFrame({'작품' : media_op_list, 
                      '설명' : media_kr_list })
media_df

,작품,설명
0,"Op. 4 Concerto 1 ""Spring"" mvt 1: Allegro","비발디의 사계 중 봄 1악장 알레그로, 요한 해리슨 바이올린"
1,"Op. 4 Concerto 1 ""Spring"" mvt 2: Largo","비발디의 사계 중 봄 2악장 라르고, 요한 해리슨 바이올린"
2,"Op. 4 Concerto 1 ""Spring"" mvt 3: Allegro","From Vivaldi's Four Seasons. John Harrison, Vi..."
3,"Op. 4 Concerto 2 ""Summer"" mvt 1: Allegro non m...","From Vivaldi's Four Seasons. John Harrison, Vi..."
4,"Op. 4 Concerto 2 ""Summer"" mvt 2: Adagio","From Vivaldi's Four Seasons. John Harrison, Vi..."
5,"Op. 4 Concerto 2 ""Summer"" mvt 3: Presto","From Vivaldi's Four Seasons. John Harrison, Vi..."
6,"Op. 4 Concerto 3 ""Autumn"" mvt 1: Allegro","From Vivaldi's Four Seasons. John Harrison, Vi..."
7,"Op. 4 Concerto 3 ""Autumn"" mvt 2: Adagio molto","From Vivaldi's Four Seasons. John Harrison, Vi..."
8,"Op. 4 Concerto 3 ""Autumn"" mvt 3: Allegro","From Vivaldi's Four Seasons. John Harrison, Vi..."
9,"Op. 4 Concerto 4 ""Winter"" mvt 1: Allegro non m...","From Vivaldi's Four Seasons. John Harrison, Vi..."


<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>